
add an env file with:
OPENAI_API_KEY = ...
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=...

to activate env cd to the project folder and run :

source env/bin/activate

In [1]:
# pip install -r requirements.txt

In [2]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [4]:
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=512)
image = encode_image("test.jpeg")




In [7]:

msg = chain.invoke(
    [   AIMessage(
        content="you will be a tutor for math students andd will help them solve the questions.the student will uopload the question and working out and you will help them by not giving the answer and having them identify the problem to for them to figure out how to solve."
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "solve"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)

It looks like you've made a mistake in the sign when you isolated the variable x. When you moved the 7 over to the other side of the equation, you correctly changed the sign from positive to negative, but then when you divided by -3, the negative sign on the 7 should have been taken into account. Can you try dividing -7 by -3 again and see what the result is? Remember that dividing a negative by a negative gives a positive result.
